# 딥러닝 (4)
## 오차역전파(Backpropagation)
오차역전파는 인공 신경망(Artificial Neural Network)을 학습시키는 데 사용되는 핵심 알고리즘입니다. 신경망의 예측 결과와 실제 정답 사이의 오차(손실, Loss)를 계산하고, 이 오차를 신경망의 출력층부터 입력층 방향으로 거꾸로 전파시키면서 각 가중치(Weight)와 편향(Bias)이 오차에 얼마나 기여했는지를 계산합니다. 이렇게 계산된 기여도(기울기, Gradient)를 사용하여 가중치와 편향을 업데이트함으로써 신경망이 점차 정확한 예측을 할 수 있도록 만듭니다.

쉽게 말해, "네트워크가 왜 틀렸는지를 파악하고, 그 틀린 정도를 각 부품(가중치, 편향)의 책임으로 나누어 할당한 뒤, 각 부품을 오차를 줄이는 방향으로 조금씩 수정하는 과정"이라고 할 수 있습니다.

* 곱셈 계층

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        dx = self.y * dout
        dy = self.x * dout
        return dx, dy

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple = MulLayer()
mul_tax = MulLayer()

apple_price = mul_apple.forward(apple, apple_num)
apple_price

200

In [3]:
mul_apple.x, mul_apple.y

(100, 2)

In [4]:
price = mul_tax.forward(apple_price, tax)
price

220.00000000000003

In [5]:
dout = 1
dapple_price, dtax = mul_tax.backward(dout)
dapple_price, dtax

(1.1, 200)

In [6]:
dapple, dapple_num = mul_apple.backward(dapple_price)
dapple, dapple_num

(2.2, 110.00000000000001)

* 덧셈 계층

In [7]:
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

* Relu 계층

In [8]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

* Sigmoid 계층

In [9]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        self.out = 1 / (1 + np.exp(-x))
        return self.out

    def backward(self, dout):
        dx = dout * (1 - self.out) * self.out
        return dx

In [10]:
def cross_entropy_error(p, r):
    delta = 1e-7
    return -np.sum(r * np.log(p + delta))

def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)

    for idx in np.ndindex(x.shape):
        tmp_val = x[idx]
        
        x[idx] = tmp_val + h
        fxh1 = f(x)

        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
    
    return grad

def softmax(x):
    if x.ndim == 1:
        c = np.max(x)
        exp_a = np.exp(x-c)
        sum_exp_a = np.sum(exp_a)
        y = exp_a / sum_exp_a
        return y
    elif x.ndim == 2:
        c = np.max(x, axis = 1).reshape(-1, 1)
        exp_a = np.exp(x - c)
        sum_exp_a = np.sum(exp_a, axis = 1).reshape(-1, 1)
        y = exp_a / sum_exp_a
        return y

* Softmax 계층

In [11]:
class SoftmaxLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss

    def backward(self, dout):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

* Affine 계층

In [12]:
class Affine:
    def __init__(self, w, b):
        self.w = w
        self.b = b
        self.x = None
        self.dw = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = x @ self.w + self.b
        return out

    def backward(self, dout):
        dx = dout @ self.w.T
        self.dw = self.x.T @ dout
        self.db = np.sum(dout, axis = 0)
        return dx

* 오차역전파 구현하기

In [13]:
from collections import OrderedDict
import numpy as np

class TwoLayerNet:
    def __init__(self, I, H, O):
        self.params = {}
        self.params['w1'] = np.random.randn(I, H)
        self.params['b1'] = np.random.randn(H)
        self.params['w2'] = np.random.randn(H, O)
        self.params['b2'] = np.random.randn(O)

        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['w1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['w2'], self.params['b2'])
        self.last_layer = SoftmaxLoss()

    def predict(self, x):
        for i in self.layers.values():
            x = i.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        accuracy = np.sum(y == t) / x.shape[0]
        return accuracy

    def numerical_gradient(self, x, t):
        loss_w = lambda w : self.loss(x, t)
        grads = {}
        grads['w1'] = numerical_gradient(loss_w, self.params['w1'])
        grads['b1'] = numerical_gradient(loss_w, self.params['b1'])
        grads['w2'] = numerical_gradient(loss_w, self.params['w2'])
        grads['b2'] = numerical_gradient(loss_w, self.params['b2'])
        return grads

    def gradient(self, x, t):
        self.loss(x, t)

        dout = 1
        dout = self.last_layer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()

        for i in layers:
            dout = i.backward(dout)

        grads = {}
        grads['w1'] = self.layers['Affine1'].dw
        grads['b1'] = self.layers['Affine1'].db
        grads['w2'] = self.layers['Affine2'].dw
        grads['b2'] = self.layers['Affine2'].db
        return grads

In [15]:
import pickle
f = open("Data/mnist.dat", 'rb')
train, test = pickle.load(f)
f.close()

train_x, train_y = train
test_x, test_y = test

train_scaled = train_x.reshape(-1, 784) / 255
test_scaled = test_x.reshape(-1, 784) / 255

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
train_y = encoder.fit_transform(train_y.reshape(-1, 1)).toarray()
test_y = encoder.fit_transform(test_y.reshape(-1, 1)).toarray()

In [ ]:
from tqdm import tqdm

net = TwoLayerNet(784, 500, 10)

train_losses = []
test_losses = []

for i in tqdm(range(10000)):
    mask = np.random.choice(60000, 1000)
    x_batch = train_scaled[mask]
    t_batch = train_y[mask]

    grad = net.gradient(x_batch, t_batch)

    for key in ('w1', 'b1', 'w2', 'b2'):
        net.params[key] -= grad[key] * 0.1

    loss = net.loss(x_batch, t_batch)
    test_loss = net.loss(test_scaled, test_y)

 17%|█████████████                                                                | 1697/10000 [06:37<35:56,  3.85it/s]

## 연습문제
1. 어떤 상품 3개의 가격이 각각 1500원이라고 할 때, 총 가격을 계산하기 위해 곱셈 계층을 사용하려 합니다.
* 순전파: 입력 x = 1500, y = 3일 때, MulLayer의 순전파 결과 out은 얼마인가요? 계산 과정을 보이세요.
* 역전파: 최종 계산된 총 가격에 대한 손실의 미분 값(상류 미분)이 dout = 0.1이라고 할 때, MulLayer의 역전파를 통해 입력 x와 y 각각에 대한 미분 값 dx와 dy는 얼마인가요? 계산 과정을 보이세요.

In [16]:
x = 1500
y = 3

dout = 0.1

dx = dout * y
dy = dout * x

dx, dy

(0.30000000000000004, 150.0)

2. 사과 가격 2000원과 바나나 가격 1500원을 더하여 총 가격을 계산하려 합니다.
* 순전파: 입력 x = 2000, y = 1500일 때, AddLayer의 순전파 결과 out은 얼마인가요? 계산 과정을 보이세요.
* 역전파: 최종 계산된 총 가격에 대한 손실의 미분 값(상류 미분)이 dout = 0.5라고 할 때, AddLayer의 역전파를 통해 입력 x와 y 각각에 대한 미분 값 dx와 dy는 얼마인가요? 계산 과정을 보이세요.

In [17]:
x = 2000
y = 1500
out = x + y

dout = 0.5

dx = dout * 1
dy = dout * 1

dx, dy

(0.5, 0.5)

3. 사과 5개 (가격 100원/개)와 오렌지 7개 (가격 150원/개)를 구매하고, 여기에 세금 20%을 적용한 최종 가격을 계산하는 상황을 가정합니다.
* 최종 가격 final_price에 대한 손실의 미분 값(상류 미분)이 dout = 1이라고 할 때, 오차역전파 과정을 따라가며 각 입력 값(사과 가격, 사과 개수, 오렌지 가격, 오렌지 개수, 세금)에 대한 최종 손실의 미분 값을 구하는 과정을 설명하고 계산해 보세요.

In [34]:
apple = 100
apple_num = 5

orange = 150
orange_num = 7

tax = 1.2      # 세금 20%

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_layer = AddLayer()           # 사과, 오렌지 더해주는 덧셈계층
mul_tax_layer = MulLayer()         # 세금 곱해주는 곱셈계층 

# 순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)

# 각각의 최송 손실의 미분값 구하라 = 역전파
dall_price, dtax = mul_tax_layer.backward(1)
dapple_price, dorange_price = add_layer.backward(dall_price)   # 사과가격을 미분, 오렌지가격을 미분

dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)

dapple, dapple_num, dorange, dorange_num, dtax 

(6.0, 120.0, 8.4, 180.0, 1550)

4. 입력 값 x = [-1, 0.5, -2, 3]이 ReLU 계층을 통과했다고 가정합니다. 순전파 결과 out을 계산하고, 상류 미분 dout = [0.2, 0.3, 0.1, 0.4]가 주어졌을 때, 역전파 결과 dx를 계산하세요. ReLU의 역전파 특징을 확인하세요.

In [36]:
x = np.array([-1, 0.5, -2, 3])

relu = Relu()
relu.forward(x)

dout = np.array([0.2, 0.3, 0.1, 0.4])
relu.backward(dout)

array([0. , 0.3, 0. , 0.4])

5. Sigmoid 계층의 순전파 결과 self.out = 0.7이라고 가정합니다. 상류 미분 dout = 0.5가 주어졌을 때, 역전파 결과 dx를 계산하세요. Sigmoid 함수의 미분 공식을 이용하여 설명해 보세요.

In [41]:
dout = 0.5
dx = dout * 0.7 * (1 - 0.7)         # 시그모이드의 역전파... dx = dout * self.out * (1 - self.out)
dx

0.10500000000000001

6. 입력 x = [1.0, 0.5] (shape (1, 2)), 가중치 행렬 W = [[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]] (shape (2, 3)), 편향 벡터 b = [0.1, 0.2, 0.3] (shape (3,))이 주어졌을 때, Affine 계층의 순전파 결과 out (shape (1, 3))을 계산하세요. 행렬 곱셈과 덧셈 과정을 보이세요.

In [44]:
x = np.array([1.0, 0.5])
W = np.array([[0.1, 0.3, 0.5], [0.2, 0.4, 0.6]])
b = np.array([0.1, 0.2, 0.3])
print(x @ W + b)

affine = Affine(W, b)
print(affine.forward(x))

[0.3 0.7 1.1]
[0.3 0.7 1.1]


7. 어떤 분류 문제에서 신경망의 출력이 Softmax 함수를 거친 확률 값 y = [0.1, 0.2, 0.7]이고, 실제 정답(원-핫 인코딩)이 t = [0, 0, 1]이라고 가정합니다. SoftmaxWithLoss 계층의 역전파를 통해 손실에 대한 Softmax 입력 값의 미분 dx를 계산하세요.

In [47]:
y = np.array([0.1, 0.2, 0.7])
t = np.array([0, 0, 1])

dx = (y - t) / 1       # 데이터가 하나밖에없으므로 1로 나눠줌 (batch size가 1)
dx

array([ 0.1,  0.2, -0.3])